In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
import collections
import copy
import numpy as np
from scipy.stats import ortho_group

In [2]:
def sample_gaussian(mu, var, n):
    return np.random.multivariate_normal(mu, var, n)

In [3]:
def sample_source_and_target_params(min_var, max_var, dim=2):
    '''
    Returns the mean and variance of eaach class in the source and target distributions.
    '''
    mus = []
    vars = []
    for i in range(4):
        mu = np.random.multivariate_normal(np.zeros(dim), np.eye(dim))
        mu = mu / np.linalg.norm(mu)
        mus.append(mu)
        var_diag = np.diag(np.random.uniform(min_var, max_var, dim))
        rot_mat = ortho_group.rvs(dim)
        var = rot_mat.T @ var_diag @ rot_mat
        vars.append(var)
    return np.array(mus), np.array(vars)

In [4]:
def sample_fixed_params(mus, vars, n, p):
    '''
    Samples n points from a mixture of two gaussians with means mus[0] and mus[1] and 
    variances vars[0] and vars[1] with probability p of being from the second gaussian.
    '''
    # sample labels 0 and 1 with 1 having probability p
    labels = np.random.binomial(1, p, n)
    # sample from the corresponding gaussian
    ones = sample_gaussian(mus[1], vars[1], n)
    zeros = sample_gaussian(mus[0], vars[0], n)
    samples = np.zeros((n, mus.shape[1]))
    samples[labels == 1] = ones[labels == 1]
    samples[labels == 0] = zeros[labels == 0]
    return torch.tensor(np.array(samples), dtype=torch.float32), torch.tensor(labels, dtype=torch.float32)

In [5]:
def sample_intermediate_params(source_mus, source_vars, target_mus, target_vars, n, b, p):
    '''
    n-> number of time steps
    b-> number of samples for each time step
    p-> probability of sampling from the second gaussian
    '''
    # sample labels 0 and 1 with 1 having probability p
    samples = []
    labels = []
    for i in range(n):
        alpha = i / n
        label = np.random.binomial(1, p[i], b)
        mus = (1 - alpha) * source_mus + alpha * target_mus
        vars = (1 - alpha) * source_vars + alpha * target_vars
        # sample from the corresponding gaussian
        ones = sample_gaussian(mus[1], vars[1], b)
        zeros = sample_gaussian(mus[0], vars[0], b)
        sample = np.zeros((b, dim))
        sample[label == 1] = ones[label == 1]
        sample[label == 0] = zeros[label == 0]
        labels.append(label)
        samples.append(sample)
    return torch.tensor(np.array(samples), dtype=torch.float32), torch.tensor(np.array(labels), dtype=torch.float32)

In [6]:
def evaluate_model(model, xs, ys):
    '''
    Returns the accuracy of the model on the given data.
    '''
    model.eval()
    with torch.no_grad():
        pred = model(xs)
        pred = pred.squeeze()
        pred[pred > 0.5] = 1
        pred[pred <= 0.5] = 0
        acc = (pred == ys).sum().item() / ys.size(0)
    return acc

In [7]:
class BayesModel(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.fc1 = nn.Linear(dim, 256)
        self.fc3 = nn.Linear(256, 1)
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = torch.sigmoid(self.fc3(x))
        return x

In [8]:
# create a linear traininable model
class LogisticRegression(nn.Module):
    def __init__(self, dim):
        super(LogisticRegression, self).__init__()
        self.fc = nn.Linear(dim, 1)

    def forward(self, x):
        x = self.fc(x)
        x = torch.sigmoid(x)
        return x

In [9]:
def train_on_source(model, xSourceTrain, ySourceTrain, epochs=1000, lr=0.01):
    '''
    Trains the input model on the input data. 
    Adam is used as the optimizer since using SGD convergence is very slow.
    '''
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        pred = model(xSourceTrain)
        loss = F.binary_cross_entropy(pred.squeeze(), ySourceTrain)
        loss.backward()
        optimizer.step()

In [10]:
def adapt1(model, xs, epochs=1000, lr=0.01):
    '''
    Unsupervised adaptation of the input model on the input data.
    '''
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.BCELoss()
    for epoch in range(epochs):
        optimizer.zero_grad()
        yPred = model(xs)
        predLabels = (yPred > 0.5).float()
        loss = criterion(yPred, predLabels.detach())
        loss.backward()
        optimizer.step()
        # if epoch % 100 == 0:
        #     print('Epoch {}, loss {}'.format(epoch, loss.item()))

In [11]:
np.random.seed(1000)
torch.manual_seed(0)

dim, nInterSteps, bTrain, bTest, bInter, pSource, pTarget = 400, 100, 10, 10000, 1, 0.5, 0.9

mus, vars = sample_source_and_target_params(0.05, 0.1, dim)         # samples mean and variance for each class in source and target
source_means = mus[:2]
source_vars = vars[:2]
target_means = mus[2:]
target_vars = vars[2:]
xSourceTrain, ySourceTrain = sample_fixed_params(source_means, source_vars, bTrain, pSource)    # samples training points from source
xSourceVal, ySourceVal = sample_fixed_params(source_means, source_vars, bTest, pSource)         # samples validation points from source
xSourceTest, ySourceTest = sample_fixed_params(source_means, source_vars, bTest, pSource)       # samples test points from source
xTargetTrain, yTargetTrain = sample_fixed_params(target_means, target_vars, bTrain, pTarget)    # samples training points from target
xTargetVal, yTargetVal = sample_fixed_params(target_means, target_vars, bTest, pTarget)         # samples validation points from target
xTargetTest, yTargetTest = sample_fixed_params(target_means, target_vars, bTest, pTarget)       # samples test points from target
xInter, yInter = sample_intermediate_params(source_means, source_vars, target_means, target_vars, nInterSteps, bInter, np.linspace(pSource, pTarget, nInterSteps))  # samples intermediate points

In [12]:
model = BayesModel(dim)
train_on_source(model, xSourceVal, ySourceVal)
sourceTestAcc = evaluate_model(model, xSourceTest, ySourceTest)
print("Bayes accuracy for source:", sourceTestAcc)

Bayes accuracy for source: 0.9904


In [13]:
model = BayesModel(dim)
train_on_source(model, xTargetVal, yTargetVal)
targetTestAcc = evaluate_model(model, xTargetTest, yTargetTest)
print("Bayes accuracy of target:", targetTestAcc)

Bayes accuracy of target: 0.9953


In [14]:
model = LogisticRegression(dim)
train_on_source(model, xSourceTrain, ySourceTrain)
sourceTrainAcc = evaluate_model(model, xSourceTrain, ySourceTrain)
sourceValAcc = evaluate_model(model, xSourceVal, ySourceVal)
sourceTestAcc = evaluate_model(model, xSourceTest, ySourceTest)
targetTrainAcc = evaluate_model(model, xTargetTrain, yTargetTrain)
targetValAcc = evaluate_model(model, xTargetVal, yTargetVal)
targetTestAcc = evaluate_model(model, xTargetTest, yTargetTest)
print("When logisic regression is trained on source:")
print('Source train acc {}, source val acc {}, source test acc {}, target train acc {}, target val acc {}, target test acc {}'.format(sourceTrainAcc, sourceValAcc, sourceTestAcc, targetTrainAcc, targetValAcc, targetTestAcc))

When logisic regression is trained on source:
Source train acc 1.0, source val acc 0.7867, source test acc 0.79, target train acc 0.4, target val acc 0.5195, target test acc 0.5176


In [15]:
model = LogisticRegression(dim)
train_on_source(model, xTargetTrain, yTargetTrain)
targetTrainAcc = evaluate_model(model, xTargetTrain, yTargetTrain)
targetValAcc = evaluate_model(model, xTargetVal, yTargetVal)
targetTestAcc = evaluate_model(model, xTargetTest, yTargetTest)
print("When logisiic regression is trained on target data:")
print('Target train acc {}, target val acc {}, target test acc {}'.format(targetTrainAcc, targetValAcc, targetTestAcc))

When logisiic regression is trained on target data:
Target train acc 1.0, target val acc 0.9161, target test acc 0.9191


In [16]:
model = LogisticRegression(dim)

train_on_source(model, xSourceTrain, ySourceTrain)

sourceTrainAcc = evaluate_model(model, xSourceTrain, ySourceTrain)
sourceValAcc = evaluate_model(model, xSourceVal, ySourceVal)
sourceTestAcc = evaluate_model(model, xSourceTest, ySourceTest)
targetTrainAcc = evaluate_model(model, xTargetTrain, yTargetTrain)
targetValAcc = evaluate_model(model, xTargetVal, yTargetVal)
targetTestAcc = evaluate_model(model, xTargetTest, yTargetTest)
print('Source train acc {}, source val acc {}, source test acc {}, target train acc {}, target val acc {}, target test acc {}'.format(sourceTrainAcc, sourceValAcc, sourceTestAcc, targetTrainAcc, targetValAcc, targetTestAcc))

accus = []
for i in range(nInterSteps):
    print("****************** INTER",i,"****************")
    xInterTrain, yInterTrain = xInter[i], yInter[i]
    adapt1(model, xInterTrain, epochs = 1000)
    sourceTrainAcc = evaluate_model(model, xSourceTrain, ySourceTrain)
    sourceValAcc = evaluate_model(model, xSourceVal, ySourceVal)
    targetTrainAcc = evaluate_model(model, xTargetTrain, yTargetTrain)
    targetValAcc = evaluate_model(model, xTargetVal, yTargetVal)
    accus.append(targetValAcc)
    print('Source train acc {}, source val acc {}, target train acc {}, target val acc {}'.format(sourceTrainAcc, sourceValAcc, targetTrainAcc, targetValAcc))
targetTestAcc = evaluate_model(model, xTargetTest, yTargetTest)
print('Target test acc {}'.format(targetTestAcc))

Source train acc 1.0, source val acc 0.7735, source test acc 0.7777, target train acc 0.4, target val acc 0.5345, target test acc 0.5368
****************** INTER 0 ****************
Source train acc 1.0, source val acc 0.7183, target train acc 0.4, target val acc 0.5619
****************** INTER 1 ****************
Source train acc 1.0, source val acc 0.7187, target train acc 0.7, target val acc 0.5897
****************** INTER 2 ****************
Source train acc 1.0, source val acc 0.7114, target train acc 0.7, target val acc 0.5836
****************** INTER 3 ****************
Source train acc 1.0, source val acc 0.7107, target train acc 0.6, target val acc 0.5959
****************** INTER 4 ****************
Source train acc 1.0, source val acc 0.7122, target train acc 0.7, target val acc 0.5989
****************** INTER 5 ****************
Source train acc 1.0, source val acc 0.6994, target train acc 0.8, target val acc 0.6229
****************** INTER 6 ****************
Source train acc 1.0,

In [17]:
def combine_inter_and_target(xInter, yInter, xTarget, yTarget):
    xInter = xInter.reshape(-1, dim)
    yInter = yInter.reshape(-1)
    xTarget = xTarget.reshape(-1, dim)
    yTarget = yTarget.reshape(-1)
    x = torch.cat((xInter, xTarget), 0)
    y = torch.cat((yInter, yTarget), 0)
    return x, y

In [28]:
skip = 50
xComb, yComb = combine_inter_and_target(xInter[skip:], yInter[skip:], xTargetTrain, yTargetTrain)

In [29]:
model = LogisticRegression(dim)
train_on_source(model, xComb, yComb)
targetTestAcc = evaluate_model(model, xTargetTest, yTargetTest)
print('When logisic regression is trained on intermediate and target data while skipping the first {} intermediate steps:'.format(skip))
print('Target test acc {}'.format(targetTestAcc))

When logisic regression is trained on intermediate and target data while skipping the first 50 intermediate steps:
Target test acc 0.9482
